In [ ]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt
from pyspark.sql import SQLContext
import json
import sys
import os
import calendar
import time
import base64
import requests
import math

#Globals for Kafka config
OUTPUTTOPIC='newOutput'
containerIP='0.0.0.0'

URL = 'http://' + containerIP + ':9443/topics/' + OUTPUTTOPIC
HEADERS = {'Content-Type': 'application/vnd.kafka.v1+json'}

#last N entries in the database
limitN = 20

#Sensors to analyze
sensors = ['PM41301', 'PM41801', 'PM107631']

sqlContext = SQLContext(sc)

def do_the_math(SensorID):
    df1 = sqlContext.sql("SELECT codigo, intensidad, velocidad, tf, dt FROM madridtraffic WHERE codigo = \'" + SensorID + "\'" + " order by ts desc limit " + str(limitN))
    df1.show()
    
    if df1.rdd.isEmpty():
        print "There's no data set for the specified sensors."
        return
    
    #mllib clustering function requires data in array and float valuesdf1
    parsedData = df1.map(lambda row: array([float(row.intensidad), float(row.velocidad)]))
    print('parsedData is mapped')
    
    #calculate mean for standardization
    mean = parsedData.mean()
    
    #calculate standard deviation
    std = parsedData.stdev()

    processedData = parsedData.map(lambda x: ((x-mean)/std))
    print 'normalized the data'

    # Build the model (cluster the data)
    clusters = KMeans.train(processedData, 2, maxIterations=10, runs=1, initializationMode="random")
    print 'clusters initilized'

    #denormalize the centers.
    centers = (clusters.centers*std)+mean
    print 'denormalized the centers'


    #midpoints-threshold
    a = (centers[0][0]+centers[1][0])/2
    b = (centers[0][1]+centers[1][1])/2

    #creates dictionary
    message = {"codigo": SensorID, "newThresholds":{"ThresholdSpeed":int(b), "ThresholdIntensity":int(a)}}
    print message
    message = base64.b64encode(str(message))

    #creates dictionary
    messages = {"records":[{"value":message}]}
    #converts dictionary into json msg
    Threshold = json.dumps(messages)
    
    #publish the message: payload is threshold and other field is topic
    r = requests.post(URL, data=Threshold, headers=HEADERS)
    print(r.text)
    
def kick_off_routine():
    #Loop through the sensors
    for i in sensors:
        do_the_math(i)

df = sqlContext.read.parquet("swift://DataServices.spark/DataServices/DataServices")

df = df.cache()
df.registerTempTable("madridtraffic")

kick_off_routine()
df.unpersist()
